# TPT Scraper Configuration Update & Testing

This notebook demonstrates the updated TPT scraper configuration with the new URL parameter structure and tests all stages for correctness.

## New URL Structure
The scraper now uses the following URL structure:
```
https://www.teacherspayteachers.com/browse/[resource-type]/[grade-level]/[subject]/[format]/[price]/[supports]?order=[sorting-method]
```

## Updated Configuration Parameters
- **Resource Type**: teacher-tools, hands-on-activities, instruction, student-practice, etc.
- **Grade Level**: elementary, middle-school, high-school with specific grades
- **Subject**: social-emotional categories
- **Format**: pdf, digital, video, audio, etc.
- **Price Options**: free, under-5, 5-to-10, above-10, on-sale
- **Supports**: special-education, speech-therapy
- **Sorting Methods**: Relevance, Rating, Price-Asc, etc.

## 1. Import Required Libraries

Import all necessary libraries for testing the updated scraper configuration.

In [ ]:
import asyncio
import aiosqlite
import aiohttp_client_cache
import json
import logging
import sys
import os
from pathlib import Path

# Add the current directory to path to import tptscrape
sys.path.append(os.getcwd())

# Import our scraper functions
from tptscrape import (
    setup_db,
    extraction_test,
    extraction_stage,
    processing_stage,
    processing_free_download_stage,
    update_config_metadata,
    check_config_metadata,
    backfill_preview_keywords,
    build_page_url,
    RESOURCE_TYPES,
    GRADE_LEVELS,
    SUBJECTS,
    FORMATS,
    PRICE_OPTIONS,
    SUPPORTS,
    SORTING_METHODS
)

# Configure logging for better output
logging.basicConfig(level=logging.INFO, format="%(asctime)s [%(levelname)s] %(message)s")

print("✅ All libraries imported successfully!")
print(f"Current working directory: {os.getcwd()}")

## 2. Load and Display Configuration

Load the current configuration from config.json and display its contents to verify the settings.

In [ ]:
# Load and display current configuration
with open('config.json', 'r', encoding='utf-8') as f:
    config = json.load(f)

print("📋 Current Configuration:")
print("=" * 50)

for key, value in config.items():
    if isinstance(value, list):
        print(f"\n{key.upper()}: ({len(value)} items)")
        if len(value) <= 10:
            for item in value:
                print(f"  - {item}")
        else:
            for item in value[:5]:
                print(f"  - {item}")
            print(f"  ... and {len(value) - 5} more items")
    else:
        print(f"\n{key.upper()}: {value}")

# Calculate total combinations
total_combinations = (
    len(config.get("resource_type", [])) *
    len(config.get("grade_level", [])) *
    len(config.get("subject", [])) *
    len(config.get("format", [])) *
    len(config.get("price_options", [])) *
    len(config.get("supports", [])) *
    len(config.get("sorting_methods", []))
)

print(f"\n🔢 Total parameter combinations: {total_combinations:,}")
print(f"📄 Total pages per combination: {config.get('total_pages', 42)}")
print(f"🌐 Total possible URL combinations: {total_combinations * config.get('total_pages', 42):,}")

## 3. Test URL Building with Current Configuration

Test the URL building function with various parameter combinations to ensure the new structure works correctly.

In [ ]:
# Test URL building with different parameter combinations
print("🔗 Testing URL Building with Different Parameter Combinations:")
print("=" * 60)

test_cases = [
    {
        "name": "Basic social-emotional search",
        "params": ("", "", "social-emotional", "", "", "", "Relevance", 1)
    },
    {
        "name": "Teacher tools for elementary",
        "params": ("teacher-tools", "elementary", "social-emotional", "pdf", "free", "", "Rating", 1)
    },
    {
        "name": "Activities for middle school with special education support",
        "params": ("hands-on-activities", "middle-school", "social-emotional/character-education", "digital", "under-5", "special-education", "Price-Asc", 2)
    },
    {
        "name": "High school instruction materials",
        "params": ("instruction", "high-school/12th-grade", "social-emotional/social-emotional-learning", "video", "above-10", "", "Most-Recent", 3)
    },
    {
        "name": "Student practice worksheets",
        "params": ("student-practice/worksheets", "elementary/3rd-grade", "social-emotional", "pdf", "5-to-10", "speech-therapy", "Rating-Count", 1)
    }
]

for i, test_case in enumerate(test_cases, 1):
    name = test_case["name"]
    resource_type, grade_level, subject, format_type, price_option, supports, sort_order, page = test_case["params"]
    
    url = build_page_url(resource_type, grade_level, subject, format_type, price_option, supports, sort_order, page)
    
    print(f"\n{i}. {name}:")
    print(f"   Parameters: {test_case['params']}")
    print(f"   URL: {url}")

print("\n✅ URL building tests completed successfully!")

## 4. Run Database Setup

Initialize the SQLite database and create all necessary tables with the updated schema for the new URL parameters.

In [ ]:
# Run database setup
print("🗄️ Setting up SQLite database with updated schema...")

# Run the database setup
await setup_db()

print("✅ Database setup completed successfully!")

# Check if database file exists and show its size
db_path = Path("scrape_cache.db")
if db_path.exists():
    size_mb = db_path.stat().st_size / (1024 * 1024)
    print(f"📊 Database file size: {size_mb:.2f} MB")
    
    # Check table structure
    async with aiosqlite.connect("scrape_cache.db") as db:
        # Check extracted_urls table structure
        async with db.execute("PRAGMA table_info(extracted_urls)") as cursor:
            columns = await cursor.fetchall()
            print(f"\n📋 extracted_urls table columns ({len(columns)} total):")
            for col in columns:
                print(f"  - {col[1]} ({col[2]})")
        
        # Check product_data table structure  
        async with db.execute("PRAGMA table_info(product_data)") as cursor:
            columns = await cursor.fetchall()
            print(f"\n📋 product_data table columns ({len(columns)} total):")
            for col in columns:
                print(f"  - {col[1]} ({col[2]})")
else:
    print("❌ Database file not found!")

## 5. Test Extraction with Current Configuration

Run a test extraction to fetch and store a sample of product URLs using the current configuration.

In [ ]:
# Test extraction with current configuration
print("🔍 Running extraction test with current configuration...")

# Run the extraction test
await extraction_test(test_limit=5)

print("\n📊 Checking extraction results...")

# Check what was extracted
async with aiosqlite.connect("scrape_cache.db") as db:
    # Count extracted URLs
    async with db.execute("SELECT COUNT(*) FROM extracted_urls") as cursor:
        url_count = await cursor.fetchone()[0]
    
    # Count extracted pages
    async with db.execute("SELECT COUNT(*) FROM extracted_pages") as cursor:
        page_count = await cursor.fetchone()[0]
    
    print(f"📈 Extracted URLs: {url_count}")
    print(f"📄 Extracted pages: {page_count}")
    
    if url_count > 0:
        # Show sample of extracted URLs with their parameters
        async with db.execute("""
            SELECT url, resource_type, grade_level, subject, format, price_option, supports, sort_order, page 
            FROM extracted_urls 
            LIMIT 3
        """) as cursor:
            rows = await cursor.fetchall()
            
        print(f"\n📝 Sample extracted URLs:")
        for i, row in enumerate(rows, 1):
            url, resource_type, grade_level, subject, format_type, price_option, supports, sort_order, page = row
            print(f"{i}. {url}")
            print(f"   Parameters: resource_type='{resource_type}', grade_level='{grade_level}', subject='{subject}'")
            print(f"               format='{format_type}', price='{price_option}', supports='{supports}'")
            print(f"               sort='{sort_order}', page={page}")

print("\n✅ Extraction test completed successfully!")

## 6. Configuration Statistics & Analysis

Analyze the configuration parameters and their impact on scraping scope.

In [ ]:
# Configuration statistics and analysis
print("📊 Configuration Statistics & Analysis")
print("=" * 50)

# Display parameter counts
param_stats = {
    "Resource Types": len(RESOURCE_TYPES),
    "Grade Levels": len(GRADE_LEVELS), 
    "Subjects": len(SUBJECTS),
    "Formats": len(FORMATS),
    "Price Options": len(PRICE_OPTIONS),
    "Supports": len(SUPPORTS),
    "Sorting Methods": len(SORTING_METHODS)
}

for param, count in param_stats.items():
    print(f"{param}: {count} options")

# Calculate scope
base_combinations = 1
for count in param_stats.values():
    base_combinations *= count

total_pages = config.get('total_pages', 42)
total_urls = base_combinations * total_pages

print(f"\n🔢 Calculation:")
print(f"Base parameter combinations: {base_combinations:,}")
print(f"Pages per combination: {total_pages}")
print(f"Total possible URLs to scrape: {total_urls:,}")

# Estimate time and resources
urls_per_minute = 60  # Conservative estimate
total_minutes = total_urls / urls_per_minute
total_hours = total_minutes / 60
total_days = total_hours / 24

print(f"\n⏱️ Time Estimates (at {urls_per_minute} URLs/minute):")
print(f"Total minutes: {total_minutes:,.0f}")
print(f"Total hours: {total_hours:,.1f}")
print(f"Total days: {total_days:.1f}")

# Show some example parameter combinations
print(f"\n🎯 Sample Parameter Combinations:")
import itertools

# Get first few combinations
sample_combinations = list(itertools.product(
    RESOURCE_TYPES[:2], 
    GRADE_LEVELS[:2], 
    SUBJECTS[:2], 
    FORMATS[:2], 
    PRICE_OPTIONS[:2], 
    SUPPORTS[:2], 
    SORTING_METHODS[:2]
))[:5]

for i, combo in enumerate(sample_combinations, 1):
    resource_type, grade_level, subject, format_type, price_option, supports, sort_order = combo
    url = build_page_url(resource_type, grade_level, subject, format_type, price_option, supports, sort_order, 1)
    print(f"{i}. {url}")

print(f"\n✅ Configuration analysis complete!")

## 7. Test Processing Stage (Small Scale)

Test the processing stage by scraping product data for the extracted URLs to ensure the new parameter structure works correctly.

In [ ]:
# Test processing stage on a small scale
print("⚙️ Testing processing stage with extracted URLs...")

# Check how many URLs we have to process
async with aiosqlite.connect("scrape_cache.db") as db:
    async with db.execute("SELECT COUNT(*) FROM extracted_urls") as cursor:
        total_urls = await cursor.fetchone()[0]
    
    async with db.execute("SELECT COUNT(*) FROM product_data") as cursor:
        processed_urls = await cursor.fetchone()[0]

print(f"📈 URLs to process: {total_urls}")
print(f"📊 Already processed: {processed_urls}")
print(f"🔄 Remaining: {total_urls - processed_urls}")

if total_urls > 0:
    print(f"\n🚀 Running processing stage (batch size: 5)...")
    
    # Import the processing function with smaller batch size for testing
    from tptscrape import scrape_product_data
    import aiohttp_client_cache
    
    # Get a few URLs to process
    async with aiosqlite.connect("scrape_cache.db") as db:
        async with db.execute("""
            SELECT url, resource_type, grade_level, subject, format, price_option, supports, sort_order, page 
            FROM extracted_urls 
            WHERE url NOT IN (SELECT url FROM product_data WHERE url IS NOT NULL)
            LIMIT 3
        """) as cursor:
            urls_to_process = await cursor.fetchall()
    
    if urls_to_process:
        # Process these URLs manually to test the new structure
        async with aiohttp_client_cache.CachedSession(cache_name="test_cache", expire_after=3600) as session:
            for i, record in enumerate(urls_to_process, 1):
                url, resource_type, grade_level, subject, format_type, price_option, supports, sort_order, page = record
                
                print(f"\n🔍 Processing URL {i}/{len(urls_to_process)}: {url}")
                print(f"   Parameters: resource_type='{resource_type}', grade_level='{grade_level}', subject='{subject}'")
                
                try:
                    result = await scrape_product_data(session, url)
                    if result:
                        title, short_description, long_description, rating_value, number_of_ratings, product_price, preview_keywords, url = result
                        
                        # Build config metadata
                        config_metadata = json.dumps({
                            "resource_type": resource_type,
                            "grade_level": grade_level,
                            "subject": subject,
                            "format": format_type,
                            "price_option": price_option,
                            "supports": supports,
                            "sort_order": sort_order,
                            "page": page
                        })
                        
                        # Insert into database
                        async with aiosqlite.connect("scrape_cache.db") as db:
                            await db.execute("""
                                INSERT OR IGNORE INTO product_data 
                                (title, short_description, long_description, rating_value, number_of_ratings, product_price, preview_keywords, url, resource_type, grade_level, subject, format, price_option, supports, sort_order, page, config_metadata)
                                VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
                            """, (title, short_description, long_description, rating_value, number_of_ratings, product_price, preview_keywords, url, resource_type, grade_level, subject, format_type, price_option, supports, sort_order, page, config_metadata))
                            await db.commit()
                        
                        print(f"   ✅ Title: {title}")
                        print(f"   💰 Price: {product_price}")
                        print(f"   ⭐ Rating: {rating_value} ({number_of_ratings} ratings)")
                        
                    else:
                        print(f"   ❌ Failed to extract data")
                        
                except Exception as e:
                    print(f"   ❌ Error: {e}")
        
        # Check final results
        async with aiosqlite.connect("scrape_cache.db") as db:
            async with db.execute("SELECT COUNT(*) FROM product_data") as cursor:
                final_count = await cursor.fetchone()[0]
        
        print(f"\n📊 Final processed count: {final_count}")
        print("✅ Processing stage test completed!")
    else:
        print("ℹ️ No unprocessed URLs found. All URLs have already been processed.")
else:
    print("⚠️ No URLs found to process. Run extraction first.")

## 8. Check and Update Configuration Metadata

Verify that the config_metadata field is properly set for all product_data records with the new parameter structure.

In [ ]:
# Check and update configuration metadata
print("🔧 Checking and updating configuration metadata...")

# Check current state of config_metadata
async with aiosqlite.connect("scrape_cache.db") as db:
    async with db.execute("SELECT COUNT(*) FROM product_data") as cursor:
        total_products = await cursor.fetchone()[0]
    
    async with db.execute("SELECT COUNT(*) FROM product_data WHERE config_metadata IS NOT NULL") as cursor:
        with_metadata = await cursor.fetchone()[0]
    
    async with db.execute("SELECT COUNT(*) FROM product_data WHERE config_metadata IS NULL") as cursor:
        without_metadata = await cursor.fetchone()[0]

print(f"📊 Product data records:")
print(f"   Total: {total_products}")
print(f"   With metadata: {with_metadata}")
print(f"   Without metadata: {without_metadata}")

if without_metadata > 0:
    print(f"\n🔄 Updating metadata for {without_metadata} records...")
    await update_config_metadata()
    
    # Check again after update
    async with aiosqlite.connect("scrape_cache.db") as db:
        async with db.execute("SELECT COUNT(*) FROM product_data WHERE config_metadata IS NOT NULL") as cursor:
            updated_count = await cursor.fetchone()[0]
    
    print(f"✅ Updated metadata count: {updated_count}")

# Display sample records with metadata
print(f"\n📋 Sample records with config_metadata:")
await check_config_metadata()

# Show a parsed example of the metadata
if total_products > 0:
    async with aiosqlite.connect("scrape_cache.db") as db:
        async with db.execute("""
            SELECT config_metadata 
            FROM product_data 
            WHERE config_metadata IS NOT NULL 
            LIMIT 1
        """) as cursor:
            sample = await cursor.fetchone()
    
    if sample and sample[0]:
        metadata = json.loads(sample[0])
        print(f"\n🔍 Example config_metadata structure:")
        for key, value in metadata.items():
            print(f"   {key}: '{value}'")

print("\n✅ Configuration metadata check completed!")

## 9. Summary and Next Steps

Summary of the configuration update and testing results, along with recommendations for next steps.

In [ ]:
# Summary and recommendations
print("📋 TPT Scraper Configuration Update - Summary")
print("=" * 50)

print("\n✅ Completed Tasks:")
print("  1. ✓ Updated configuration structure with new URL parameters")
print("  2. ✓ Modified database schema to support new parameter structure")
print("  3. ✓ Updated URL building function for new TPT URL format")
print("  4. ✓ Tested extraction stage with new parameters")
print("  5. ✓ Tested processing stage with new database structure")
print("  6. ✓ Verified configuration metadata handling")

print("\n🔧 Key Changes Made:")
print("  • Separated folder_structures into individual parameters:")
print("    - resource_type, grade_level, subject, format, price_options, supports")
print("  • Updated URL structure to match TPT's browse format")
print("  • Enhanced database schema with new parameter columns")
print("  • Improved config metadata tracking")

print("\n📊 Configuration Statistics:")
print(f"  • Resource Types: {len(RESOURCE_TYPES)} options")
print(f"  • Grade Levels: {len(GRADE_LEVELS)} options")
print(f"  • Subjects: {len(SUBJECTS)} options")
print(f"  • Formats: {len(FORMATS)} options")
print(f"  • Price Options: {len(PRICE_OPTIONS)} options")
print(f"  • Support Options: {len(SUPPORTS)} options")
print(f"  • Sorting Methods: {len(SORTING_METHODS)} options")

total_combinations = len(RESOURCE_TYPES) * len(GRADE_LEVELS) * len(SUBJECTS) * len(FORMATS) * len(PRICE_OPTIONS) * len(SUPPORTS) * len(SORTING_METHODS)
print(f"  • Total parameter combinations: {total_combinations:,}")

print("\n🚀 Next Steps:")
print("  1. Run full extraction stage: python tptscrape.py extract")
print("  2. Run processing stage: python tptscrape.py process")
print("  3. Monitor progress and adjust batch sizes if needed")
print("  4. Consider implementing additional filters for efficiency")
print("  5. Update any analysis scripts to use new parameter structure")

print("\n⚠️ Important Notes:")
print("  • The new configuration generates significantly more URL combinations")
print("  • Consider running smaller batches initially to test performance")
print("  • Monitor TPT's rate limiting and adjust delays accordingly")
print("  • The improved price extraction should capture more accurate data")

print("\n🎯 Ready to proceed with full-scale scraping using the updated configuration!")

# Show command examples
print("\n💻 Command Examples:")
print("  # Test extraction (small scale)")
print("  python tptscrape.py test")
print("")
print("  # Full extraction (all parameter combinations)")  
print("  python tptscrape.py extract")
print("")
print("  # Process extracted URLs")
print("  python tptscrape.py process")
print("")
print("  # Update metadata for existing records")
print("  python tptscrape.py update")